In [ ]:
#Code for finetuning and uploading to HugginFace, using HF transformers

In [ ]:
#code taken from https://github.com/huggingface/community-events/blob/main/whisper-fine-tuning-event/README.md#google-colab
#and https://huggingface.co/learn/audio-course/en/chapter5/fine-tuning

In [ ]:
#First link notebook to HuggingFace
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
#Next, make sure required packages are installed
!add-apt-repository -y ppa:jonathonf/ffmpeg-4
!apt update
!apt install -y ffmpeg

In [ ]:
#Packages, cont'd.
!pip install git+https://github.com/huggingface/datasets
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.3.0
!pip install jiwer
!pip install gradio
!pip install more-itertools

In [ ]:
#Now load dataset, using streaming
#NB: using the HF whisper fine tune event colab notebook here
#Not the Dhivehi one, which was used in No.1
from datasets import interleave_datasets, load_dataset

def load_streaming_dataset(dataset_name, split, **kwargs):
    if "+" in split:
        # load multiple splits separated by the `+` symbol *with* streaming mode
        dataset_splits = [load_dataset(dataset_name, split=split_name, streaming=True, **kwargs) for split_name in split.split("+")]
        # interleave multiple splits to form one dataset
        interleaved_dataset = interleave_datasets(dataset_splits)
        return interleaved_dataset
    else:
        # load a single split *with* streaming mode
        dataset = load_dataset(dataset_name, split=split, streaming=True, **kwargs)
        return dataset

In [ ]:
#use the above defined function on sjd dataset:
from datasets import IterableDatasetDict

raw_datasets = IterableDatasetDict()

raw_datasets["train"] = load_streaming_dataset("magnolia-resin/sjd-asr-v2", split="train")
raw_datasets["test"] = load_streaming_dataset("magnolia-resin/sjd-asr-v2", split="test")

In [ ]:
#prepare processor
from transformers import WhisperProcessor

#try English first. If successful, try Russian and Finnish
processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Russian", task="transcribe")

In [ ]:
#preprocess data
#check audio sampling rate (will be 41000)
raw_datasets["train"].features

In [ ]:
#preprocess data
#correct sampling rate to 16000
from datasets import Audio

raw_datasets = raw_datasets.cast_column("audio", Audio(sampling_rate=16000))

In [ ]:
#preprocess data
#leave case and punctuation as is (sjd set already preprocessed)
from transformers.models.whisper.english_normalizer import BasicTextNormalizer

do_lower_case = False
do_remove_punctuation = False

normalizer = BasicTextNormalizer()

In [ ]:
#preprocess data
#load and resample audio
#compute log-Mel spectrogram
#encode transcriptions as label ids w/ tokenizer

def prepare_dataset(batch):
    # load and (possibly) resample audio datato 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = processor.feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]
    # compute input length of audio sample in seconds
    batch["input_length"] = len(audio["array"]) / audio["sampling_rate"]

    # optional pre-processing steps
    transcription = batch["transcription"]
    if do_lower_case:
        transcription = transcription.lower()
    if do_remove_punctuation:
        transcription = re.sub(punctuation_to_remove_regex, " ", transcription).strip()

    # encode target text to label ids
    batch["labels"] = processor.tokenizer(transcription).input_ids
    return batch

In [ ]:
#apply prepare_datasest function
vectorized_datasets = raw_datasets.map(prepare_dataset, remove_columns=list(next(iter(raw_datasets.values())).features)).with_format("torch")

In [ ]:
#training and evaluation

In [ ]:
#defining data collator
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [ ]:
#initialise data collator
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [ ]:
#Import/set WER for eval.
import evaluate

metric = evaluate.load("wer")

In [ ]:
# evaluate with the 'normalised' WER
do_normalize_eval = True

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    if do_normalize_eval:
        pred_str = [normalizer(pred) for pred in pred_str]
        label_str = [normalizer(label) for label in label_str]
        # filtering step to only evaluate the samples that correspond to non-zero references:
        pred_str = [pred_str[i] for i in range(len(pred_str)) if len(label_str[i]) > 0]
        label_str = [label_str[i] for i in range(len(label_str)) if len(label_str[i]) > 0]

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)
    return {"wer":wer}

In [ ]:
#load pretrained checkpoint
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

In [ ]:
from transformers import GenerationConfig

# Create a GenerationConfig object
generation_config = GenerationConfig.from_pretrained("openai/whisper-small")  # Load default config from the model

# Set desired generation parameters
generation_config.use_cache = False
generation_config.suppress_tokens = []
generation_config.forced_decoder_ids = None
#specify language here:
generation_config.language = "Russian"

# Assign the generation config to your model
model.generation_config = generation_config

In [ ]:
#define training config.
#set args
from transformers import Seq2SeqTrainingArguments

#use args from Dhivehi tutorial
training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-sjd-v6",  # your repo name
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,
    learning_rate=1e-5,
    lr_scheduler_type="constant_with_warmup",
    warmup_steps=50,
    max_steps=500,
    gradient_checkpointing=True,
    fp16=True,
    fp16_full_eval=True,
    eval_strategy="steps",
    per_device_eval_batch_size=16,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=500,
    eval_steps=500,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)

In [ ]:
#import seq2seq trainer
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=vectorized_datasets["train"],
    eval_dataset=vectorized_datasets["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    processing_class=processor,
)

In [ ]:
#save model and processor to output directory prior to training
model.save_pretrained(training_args.output_dir)
processor.save_pretrained(training_args.output_dir)

In [ ]:
#launch trainer
trainer.train()

In [ ]:
#set key word arguments
kwargs = {
    "dataset_tags": "magnolia-resin/sjd-asr-v2",
    "dataset": "Kildin Sami V2 Dataset",  # a 'pretty' name for the training dataset
    "model_name": "Whisper Small Sjd - Rus Base - V6",  # a 'pretty' name for your model
    "finetuned_from": "openai/whisper-small",
    "tasks": "automatic-speech-recognition",
}

In [ ]:
#use to upload training results to HF
trainer.push_to_hub(**kwargs)